In [17]:
from geopy.exc import GeocoderTimedOut
from geopy.exc import GeocoderUnavailable
from geopy.geocoders import Nominatim
import glob
from IPython.display import display
import json
import nltk
from nltk.tokenize import TweetTokenizer
import pandas as pd
import re
from string import punctuation

## Loading Data

### Load Tweets

In [3]:
# Load tweets as JSON
allTweets_json = {'tweets': []}
for file in glob.glob('Your/Data/Path/*'):
    with open(file) as data_file:
        json_data = json.load(data_file)
        for tweet in json_data['tweet_data']:
            allTweets_json['tweets'].append(tweet)
allTweets_json

{'tweets': [{'author_id': '697374646614781952',
   'text': 'RT @AWalkerSmith7: @AaccCeta @ArchbishopSapit @LesmoreGEzkiel @DrMwaniki @fmwombeki Thanking God 4 this historic moment n church history n Kenya! 2 God b the glory! Ashe! Amen! @Oikoumene @AaccCeta @AdNAUbuntu @JaCouncilChurch @ncccusa @iamepiscopalian @GETIWCC @WCCpaween @AnglicansEndGBV @_AfricanUnion @NCIC_Kenya @bawruk @UN_Women @PaulineMuchina @proctorconf',
   'lang': 'en',
   'referenced_tweets': [{'type': 'retweeted', 'id': '1437029861676228618'}],
   'reply_settings': 'everyone',
   'entities': {'mentions': [{'start': 3,
      'end': 17,
      'username': 'AWalkerSmith7',
      'id': '1868363888'},
     {'start': 19,
      'end': 28,
      'username': 'AaccCeta',
      'id': '697374646614781952'},
     {'start': 29,
      'end': 45,
      'username': 'ArchbishopSapit',
      'id': '798412484742959105'},
     {'start': 46, 'end': 61, 'username': 'LesmoreGEzkiel', 'id': '350241176'},
     {'start': 62,
      'end': 72,


In [4]:
# Convert JSON tweets to DataFrame
data = [pd.json_normalize(allTweets_json['tweets'])]
df = pd.concat(data, axis=0, ignore_index=True)

# Parse datetime column to datetime
df.created_at = pd.to_datetime(df.created_at)
df

,author_id,text,lang,referenced_tweets,reply_settings,conversation_id,id,created_at,entities.mentions,entities.annotations,...,entities.hashtags,entities.urls,in_reply_to_user_id,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,withheld.copyright,withheld.country_codes,withheld.scope,entities.cashtags
0,697374646614781952,RT @AWalkerSmith7: @AaccCeta @ArchbishopSapit ...,en,"[{'type': 'retweeted', 'id': '1437029861676228...",everyone,1437051433480753156,1437051433480753156,2021-09-12 13:52:15+00:00,"[{'start': 3, 'end': 17, 'username': 'AWalkerS...","[{'start': 93, 'end': 95, 'probability': 0.919...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,697374646614781952,RT @sceptwic: Another important step in road t...,en,"[{'type': 'retweeted', 'id': '1437043141979295...",everyone,1437051325255192582,1437051325255192582,2021-09-12 13:51:49+00:00,"[{'start': 3, 'end': 12, 'username': 'sceptwic...",NaN,...,"[{'start': 48, 'end': 73, 'tag': 'JusticeDigni...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,697374646614781952,Congratulations Ven Rose Okeno ACK Diocese of ...,en,NaN,everyone,1436979065114800131,1436979065114800131,2021-09-12 09:04:41+00:00,NaN,"[{'start': 16, 'end': 29, 'probability': 0.516...",...,NaN,"[{'start': 213, 'end': 236, 'url': 'https://t....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,697374646614781952,Consecration Service of Ven Rose Okeno ACK Dio...,en,NaN,everyone,1436975615480041473,1436975615480041473,2021-09-12 08:50:59+00:00,NaN,"[{'start': 124, 'end': 147, 'probability': 0.5...",...,NaN,"[{'start': 239, 'end': 262, 'url': 'https://t....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,697374646614781952,We are looking forward to today’s Consecration...,en,"[{'type': 'replied_to', 'id': '143695708915549...",everyone,1436957089155493889,1436957092787757056,2021-09-12 07:37:22+00:00,NaN,"[{'start': 116, 'end': 124, 'probability': 0.6...",...,NaN,NaN,697374646614781952,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308837,2193947323,In our New Year Message to the Nation we proff...,en,"[{'type': 'quoted', 'id': '1082850893807144962'}]",everyone,1082937561537044480,1082937561537044480,2019-01-09 09:50:09+00:00,NaN,NaN,...,NaN,"[{'start': 271, 'end': 294, 'url': 'https://t....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308838,2193947323,RT @NewsDayZimbabwe: THE @zccinzim has launche...,en,"[{'type': 'retweeted', 'id': '1082932233500065...",everyone,1082934512777117696,1082934512777117696,2019-01-09 09:38:02+00:00,"[{'start': 3, 'end': 19, 'username': 'NewsDayZ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308839,2193947323,We can choose the route of engagement or that ...,en,NaN,everyone,1082904479526240256,1082904479526240256,2019-01-09 07:38:41+00:00,"[{'start': 211, 'end': 227, 'username': 'NewsD...",NaN,...,NaN,"[{'start': 183, 'end': 206, 'url': 'https://t....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308840,2193947323,"For many Zimbabweans, 2019 has come amidst muc...",en,NaN,everyone,1082548265617371137,1082548265617371137,2019-01-08 08:03:13+00:00,"[{'start': 187, 'end': 199, 'username': 'EFZZI...",NaN,...,NaN,"[{'start': 264, 'end': 287, 'url': 'https://t....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Load user data

In [5]:
# Load user data as JSON
allUsers_json = {'users': []}
for file in glob.glob('Your/Data/Path/*'):
    with open(file) as data_file:
        json_data = json.load(data_file)
        allUsers_json['users'].append(json_data['user_data'])
allUsers_json

{'users': [{'username': 'AaccCeta',
   'location': 'Waiyaki Way, Westlands',
   'created_at': '2016-02-10T11:00:47.000Z',
   'id': '697374646614781952',
   'url': 'https://t.co/ekmVD9VzeT',
   'public_metrics': {'followers_count': 1565,
    'following_count': 666,
    'tweet_count': 2795,
    'listed_count': 16},
   'name': 'AACC-CETA',
   'verified': False,
   'entities': {'url': {'urls': [{'start': 0,
       'end': 23,
       'url': 'https://t.co/ekmVD9VzeT',
       'expanded_url': 'http://www.aacc-ceta.org',
       'display_url': 'aacc-ceta.org'}]}},
   'description': 'The All Africa Conference of Churches (AACC) is a fellowship of Christian churches in Africa which confess Christ as God and Saviour according to the Scriptures'},
  {'verified': False,
   'name': 'ACNA',
   'description': 'The Anglican Church in North America - Reaching North America with the transforming love of Jesus Christ',
   'public_metrics': {'followers_count': 9023,
    'following_count': 1796,
    'tweet_cou

In [6]:
# Convert JSON data to dataframe
data = [pd.json_normalize(allUsers_json['users'])]
udf = pd.concat(data, axis=0, ignore_index=True)

# Parse time to datetime object
udf.created_at = pd.to_datetime(df.created_at)
udf

,username,location,created_at,id,url,name,verified,description,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,entities.url.urls,entities.description.hashtags,entities.description.mentions,entities.description.urls
0,AaccCeta,"Waiyaki Way, Westlands",2021-09-12 13:52:15+00:00,697374646614781952,https://t.co/ekmVD9VzeT,AACC-CETA,False,The All Africa Conference of Churches (AACC) i...,1565,666,2795,16,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,NaN,NaN
1,The_ACNA,NaN,2021-09-12 13:51:49+00:00,87371572,https://t.co/JQ6BCfucwT,ACNA,False,The Anglican Church in North America - Reachin...,9023,1796,3393,123,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,NaN,NaN
2,AngliAlliance,Anglican Communion Office,2021-09-12 09:04:41+00:00,903497472,http://t.co/v1ER3owGiA,Anglican Alliance,False,The Anglican Alliance brings together churches...,2291,1195,3599,52,"[{'start': 0, 'end': 22, 'url': 'http://t.co/v...",NaN,NaN,NaN
3,AnglicanCentre,Rome,2021-09-12 08:50:59+00:00,761714520,https://t.co/RHasTwWF4t,Anglican Centre Rome,False,The home of Anglican ecumenical engagement wit...,2681,839,1733,41,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,NaN,NaN
4,AnglicaninKenya,"Nairobi, Kenya",2021-09-12 07:37:22+00:00,1020255964044328961,https://t.co/UQQV7vquJv,Anglican Church of Kenya,False,This is the Official Twitter account of the An...,1095,185,567,3,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,WorldVisionWARO,"Dakar, Senegal",2021-05-28 15:19:41+00:00,830807449,https://t.co/9aTGZF68ZQ,World Vision West Africa,False,"In West Africa, @WorldVision is partnering wit...",2245,563,2088,44,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,"[{'start': 16, 'end': 28, 'username': 'WorldVi...",NaN
122,WorldVision,NaN,2021-05-26 13:16:51+00:00,14086764,https://t.co/uYc0FUK29s,World Vision,True,"We are a Christian relief, development and adv...",1112426,32401,11693,4743,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,NaN,NaN
123,WSCFEurope,All over Europe,2021-05-26 04:32:48+00:00,196547848,http://t.co/CE1JhmTpnt,WSCF Europe,False,The World Student Christian Federation (WSCF) ...,1026,650,2100,26,"[{'start': 0, 'end': 22, 'url': 'http://t.co/C...",NaN,NaN,NaN
124,ELCAYAGM,"Chicago, IL | Worldwide",2021-05-25 12:14:47+00:00,1511695525,,ELCA YAGM,False,The Young Adults in Global Mission program inv...,2139,444,1682,30,NaN,NaN,NaN,NaN


## User DataFrame Modification

### Get geolocations from location info 

In [7]:
# Classes
class counter:
    def __init__(self, count):
        self.count = count

# Functions
# --- return all TUA with location info in profile
def has_loc(udf):
    udf_no_loc = udf[udf.location.isna()]
    return(udf.drop(udf_no_loc.index, axis=0))

# --- Try to identify location with Nominatim
def get_geolocation(geo_locator, location):
    try:
        geolocation = geo_locator.geocode(location, timeout=1, addressdetails=True)
        
        if geolocation is None:
            geolocation = manual_backup(geo_locator, location)

        return geolocation

    except GeocoderTimedOut:
        return -1

    except GeocoderUnavailable:
        return manual_backup(geo_locator, location)

# --- manual backup for conversion of locations
def manual_backup(geo_locator, location):
    print('Location: "' + str(location) + '" can\'t be identified. If manually possible please enter location. Else enter 0.')
    geolocation = input()

    try:
        geolocation = geo_locator.geocode(geolocation, timeout=1, addressdetails=True)
    except GeocoderTimedOut:
        counter_unknown_locs.count += 1

    return(geolocation)

# --- convert location info to geolocation
def get_geolocations(geo_locator, data):
    geolocation_data = [[],[]]

    # Get longitude und altitude for TUA
    for location in has_loc(udf).location:
        geolocation_data[0].append(location)

        # Try Nominatim to automatically get location information
        geolocation = get_geolocation(geo_locator, location)

        if geolocation and geolocation != -1:
            geolocation_data[1].append(geolocation)
        elif geolocation == -1 or geolocation is None:
            counter_unknown_locs.count += 1
        else:
            sys.exit('😐 Error due to false input!')

    return geolocation_data

# Main
counter_unknown_locs = counter(0)
geo_locator = Nominatim(user_agent="VW_Twitter_Account_Mapping")

geolocation_data = get_geolocations(geo_locator, udf)
geolocation_data

Location: "London, Geneva and New York" can't be identified. If manually possible please enter location. Else enter 0.
Location: "UK - Salesian Link Office" can't be identified. If manually possible please enter location. Else enter 0.
Location: "Washington, DC | New York, NY" can't be identified. If manually possible please enter location. Else enter 0.
Location: "ELCA Advocacy - Washington, DC" can't be identified. If manually possible please enter location. Else enter 0.
Location: "All around the world" can't be identified. If manually possible please enter location. Else enter 0.
Location: "Presbyterian Church (U.S.A.)" can't be identified. If manually possible please enter location. Else enter 0.
Location: "Manila | Cebu | Davao PH" can't be identified. If manually possible please enter location. Else enter 0.
Location: "Plot 15B Nakasero, Kampala" can't be identified. If manually possible please enter location. Else enter 0.
Location: "All over Europe" can't be identified. If man

[['Waiyaki Way, Westlands',
  'Anglican Communion Office',
  'Rome',
  'Nairobi, Kenya',
  'London, Geneva and New York',
  'Global South',
  'Washington, DC',
  'Cambridge',
  'Winnipeg, Manitoba',
  'Vatikanstadt',
  'Canning Town, East London',
  'Australia',
  'Brussels',
  'Jordan',
  'Lebanon',
  'Archdiocese of Manila',
  'Abuja, Nigeria',
  'Switzerland',
  'Vatican ',
  'Richmond, VA',
  'Vatican City',
  'UK - Salesian Link Office',
  'Canada',
  'UK',
  'UK',
  'Washington, DC | New York, NY',
  'Ireland',
  'Geneva',
  'Brussels',
  'Colorado | Worldwide',
  'Brussels, Strasbourg',
  'Casper, Wyoming',
  'EU, ',
  'London',
  'Jerusalem',
  'Geneva, Switzerland',
  'Geneva, Switzerland',
  'Harare, Zimbabwe',
  'Philippines',
  'ELCA Advocacy - Washington, DC',
  'Chicago, IL',
  'Chicago, IL',
  'Bologna, Emilia Romagna',
  'Vereinigtes Königreich',
  'London, UK',
  'London, England',
  'United States',
  'Atlanta, GA',
  'Texas, USA',
  'Berkeley, CA',
  'Città del Vatic

In [8]:
# Functions
# --- Get Countries for all TUA
def get_country(geolocation_data):
    country_list = [[], []]

    for geolocation in geolocation_data[1]:
        try:
            country = geolocation.raw['address']['country']
        except KeyError:
            print(f'The following geocode can\'t be assigned to a county: {str(geolocation.raw)}' + '\n If country can be identified manually please enter. Else enter 0.')
            country = input()

            if country == 0:
                continue

        geolocation = geo_locator.geocode(country, timeout=1)

        country_list[0].append(geolocation)
        country_list[1].append(country)

    return country_list

# Main
country_list = get_country(geolocation_data)
country_list

[[Location(Kenya, (1.4419683, 38.4313975, 0.0)),
  Location(United Kingdom, (54.7023545, -3.2765753, 0.0)),
  Location(Italia, (42.6384261, 12.674297, 0.0)),
  Location(Kenya, (1.4419683, 38.4313975, 0.0)),
  Location(United Kingdom, (54.7023545, -3.2765753, 0.0)),
  Location(Nederland, (52.15517, 5.38721, 0.0)),
  Location(United States, (39.7837304, -100.445882, 0.0)),
  Location(United Kingdom, (54.7023545, -3.2765753, 0.0)),
  Location(Canada, (61.0666922, -107.991707, 0.0)),
  Location(Civitas Vaticana - Città del Vaticano, (41.903411, 12.4528527, 0.0)),
  Location(United Kingdom, (54.7023545, -3.2765753, 0.0)),
  Location(Australia, (-24.7761086, 134.755, 0.0)),
  Location(België / Belgique / Belgien, (50.6402809, 4.6667145, 0.0)),
  Location(الأردن, (31.1667049, 36.941628, 0.0)),
  Location(لبنان, (33.8750629, 35.843409, 0.0)),
  Location(Pilipinas / Philippines, (12.7503486, 122.7312101, 0.0)),
  Location(Nigeria, (9.6000359, 7.9999721, 0.0)),
  Location(Schweiz/Suisse/Svizzera

In [9]:
# Functions
# --- Prepare User Dataframe for analysis
def tweak_user(udf):
    # Drop columns not needed for analysis
    tudf = (udf
            .drop(columns=[c for c in df.columns if 'created_at' in c])
            .drop(columns=['username', 'description', 'url', 'public_metrics.listed_count', 
                           'entities.url.urls', 'entities.description.hashtags', 
                           'entities.description.mentions', 'entities.description.urls'])
           )
    
    # Merge with geolocation information
    merged_df = (pd.Series(country_list[1])
                 .rename('country')
                 .to_frame()
                 .merge(pd.Series(geolocation_data[0])
                 .rename('location')
                 .to_frame()
                 , left_index=True, right_index=True)
            )

    return(tudf
           .merge(merged_df, left_on='location', right_on='location', how='left')
           .sort_values(by=['name'])
           .drop_duplicates(subset=['name'])
          )

tudf = tweak_user(udf)
with pd.option_context('display.max_colwidth', 280):
    display(tudf)

,location,id,name,verified,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,country
0,"Waiyaki Way, Westlands",697374646614781952,AACC-CETA,False,1565,666,2795,Kenya
1,NaN,87371572,ACNA,False,9023,1796,3393,NaN
2,Anglican Communion Office,903497472,Anglican Alliance,False,2291,1195,3599,United Kingdom
3,Rome,761714520,Anglican Centre Rome,False,2681,839,1733,Italia
4,"Nairobi, Kenya",1020255964044328961,Anglican Church of Kenya,False,1095,185,567,Kenya
...,...,...,...,...,...,...,...,...
166,"Federal Way, WA",11859822,World Vision USA,True,554386,14283,92602,Sénégal
164,"Plot 15B Nakasero, Kampala",759998299,World Vision Uganda,False,4278,580,3375,United Kingdom
167,"Dakar, Senegal",830807449,World Vision West Africa,False,2245,563,2088,United States
157,"Addis Ababa, Ethiopia",2231308272,WorldVision Ethiopia,False,1958,178,420,België / Belgique / Belgien


## Tweet DataFrame Modification (NLP)

In [12]:
TWEET_TOKENIZER = TweetTokenizer(preserve_case=False,
                                 strip_handles=True, reduce_len=False)
STOPWORDS = (set(nltk.corpus.stopwords.words("english")) |
             {"...", '…', '•', '’', "com"} |
             set(punctuation))

# Functions
# --- Replace URLs in `string` with text of `replacement`
def replace_urls(string, replacement=None):
    if replacement is None:
        replacement = "<-URL->"
    pattern = re.compile('(https?://)?(\w*[.]\w+)+([/?=&]+\w+)*')
    return re.sub(pattern, replacement, string)

# --- Tokenize text and remove punctuation/stopwords etc
def tokenizer(tweet_text, custom_words=None):
    text = (replace_urls(tweet_text))
    tokens = TWEET_TOKENIZER.tokenize(text.lower())
    tokens = (token for token in tokens if token not in punctuation)
    tokens = (token for token in tokens if token.islower())
    tokens = (token for token in tokens if token not in STOPWORDS)
    tokens = (token for token in tokens if len(token) >= 3)
    if custom_words:
        tokens = (token for token in tokens if token not in custom_words)
    return list(tokens)

# Main
df = df.assign(text_processed=df['text'].apply(lambda t: tokenizer(t)))
df

,author_id,text,lang,referenced_tweets,reply_settings,conversation_id,id,created_at,entities.mentions,entities.annotations,...,entities.urls,in_reply_to_user_id,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,withheld.copyright,withheld.country_codes,withheld.scope,entities.cashtags,text_processed
0,697374646614781952,RT @AWalkerSmith7: @AaccCeta @ArchbishopSapit ...,en,"[{'type': 'retweeted', 'id': '1437029861676228...",everyone,1437051433480753156,1437051433480753156,2021-09-12 13:52:15+00:00,"[{'start': 3, 'end': 17, 'username': 'AWalkerS...","[{'start': 93, 'end': 95, 'probability': 0.919...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[thanking, god, historic, moment, church, hist..."
1,697374646614781952,RT @sceptwic: Another important step in road t...,en,"[{'type': 'retweeted', 'id': '1437043141979295...",everyone,1437051325255192582,1437051325255192582,2021-09-12 13:51:49+00:00,"[{'start': 3, 'end': 12, 'username': 'sceptwic...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[another, important, step, road, #justicedigni..."
2,697374646614781952,Congratulations Ven Rose Okeno ACK Diocese of ...,en,NaN,everyone,1436979065114800131,1436979065114800131,2021-09-12 09:04:41+00:00,NaN,"[{'start': 16, 'end': 29, 'probability': 0.516...",...,"[{'start': 213, 'end': 236, 'url': 'https://t....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[congratulations, ven, rose, okeno, ack, dioce..."
3,697374646614781952,Consecration Service of Ven Rose Okeno ACK Dio...,en,NaN,everyone,1436975615480041473,1436975615480041473,2021-09-12 08:50:59+00:00,NaN,"[{'start': 124, 'end': 147, 'probability': 0.5...",...,"[{'start': 239, 'end': 262, 'url': 'https://t....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[consecration, service, ven, rose, okeno, ack,..."
4,697374646614781952,We are looking forward to today’s Consecration...,en,"[{'type': 'replied_to', 'id': '143695708915549...",everyone,1436957089155493889,1436957092787757056,2021-09-12 07:37:22+00:00,NaN,"[{'start': 116, 'end': 124, 'probability': 0.6...",...,NaN,697374646614781952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[looking, forward, today, consecration, enthro..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308837,2193947323,In our New Year Message to the Nation we proff...,en,"[{'type': 'quoted', 'id': '1082850893807144962'}]",everyone,1082937561537044480,1082937561537044480,2019-01-09 09:50:09+00:00,NaN,NaN,...,"[{'start': 271, 'end': 294, 'url': 'https://t....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[new, year, message, nation, proffered, areas,..."
308838,2193947323,RT @NewsDayZimbabwe: THE @zccinzim has launche...,en,"[{'type': 'retweeted', 'id': '1082932233500065...",everyone,1082934512777117696,1082934512777117696,2019-01-09 09:38:02+00:00,"[{'start': 3, 'end': 19, 'username': 'NewsDayZ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[launched, fresh, bid, broker, dialogue, presi..."
308839,2193947323,We can choose the route of engagement or that ...,en,NaN,everyone,1082904479526240256,1082904479526240256,2019-01-09 07:38:41+00:00,"[{'start': 211, 'end': 227, 'username': 'NewsD...",NaN,...,"[{'start': 183, 'end': 206, 'url': 'https://t....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[choose, route, engagement, conflict, route, i..."
308840,2193947323,"For many Zimbabweans, 2019 has come amidst muc...",en,NaN,everyone,1082548265617371137,1082548265617371137,2019-01-08 08:03:13+00:00,"[{'start': 187, 'end': 199, 'username': 'EFZZI...",NaN,...,"[{'start': 264, 'end': 287, 'url': 'https://t....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[many, zimbabweans, come, amidst, much, hardsh..."


## Save DataFrames

In [16]:
# Save User DataFrame
tudf.to_pickle('Your/Data/Path/tweaked_user_dataframe.pkl')

# Save Tweet DataFrame
df.to_pickle('Your/Data/Path/tweet_dataframe.pkl')

# Check if saving worked by displaying on of the DataFrames
nudf = pd.read_pickle('Your/Data/Path/tweaked_user_dataframe.pkl')
nudf

,location,id,name,verified,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,country
0,"Waiyaki Way, Westlands",697374646614781952,AACC-CETA,False,1565,666,2795,Kenya
1,NaN,87371572,ACNA,False,9023,1796,3393,NaN
2,Anglican Communion Office,903497472,Anglican Alliance,False,2291,1195,3599,United Kingdom
3,Rome,761714520,Anglican Centre Rome,False,2681,839,1733,Italia
4,"Nairobi, Kenya",1020255964044328961,Anglican Church of Kenya,False,1095,185,567,Kenya
...,...,...,...,...,...,...,...,...
166,"Federal Way, WA",11859822,World Vision USA,True,554386,14283,92602,Sénégal
164,"Plot 15B Nakasero, Kampala",759998299,World Vision Uganda,False,4278,580,3375,United Kingdom
167,"Dakar, Senegal",830807449,World Vision West Africa,False,2245,563,2088,United States
157,"Addis Ababa, Ethiopia",2231308272,WorldVision Ethiopia,False,1958,178,420,België / Belgique / Belgien
